#**Rapid classification of amorphous-crystalline phase using artificial neural networks and resistivity mapping Ⅱ**

This script is to reproduce the models described in the article entitled of "Electrical Resistivity as a Descriptor for Classification of Amorphous-Crystalline Phases of Alloys," authored by Daegun You, Haitao Zhang, Shraddha Ganorkar, Taeyeop Kim, Jan Schroers, Joost J. Vlassak, and Dongwoo Lee. This is exported version for web-based running, so some minor discrepancies occur with compared to the original script (refer to the original page on github). First, you should download the matrix data file named of 'ML-R-MG.mat' in the original page. Then, it will ask you to upload the matrix again in this page.

In [3]:
!pip install mat4py
import numpy
import numpy as np
from mat4py import loadmat
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def scaler(X):
    X = (X-np.mean(X, axis=0))/np.std(X, axis=0, ddof=1)
    return X

from google.colab import files
file_uploaded = files.upload()

Saving ML-R-MG.mat to ML-R-MG.mat


Load training/test datasets as matrix that includes all features and output (already divided into two sets randomly 20% selected test data)
###### 0-absolute resistivity, 1-modified resistivity
###### 2-average atomic radii, 3-atomic mismatch
###### 4-heat of mixing, 5-average electronegativity
###### 6-electronegativity difference, 7-number of atom
###### 8-Amorphous/Crystalline label (1/0)

In [4]:
data = loadmat('ML-R-MG.mat')
data_training = data['trainingset_HTEwithLiter']
data_test = data['testset_HTEwithLiter']
data_training = np.asarray(data_training, dtype=np.float64)
data_test = np.asarray(data_test, dtype=np.float64)

Scaling input features on the total domain (training+test)

In [5]:
X = scaler(np.vstack((data_training[:, :8], data_test[:, :8])))

Define training and test input, and their output as well
Input example below can be varied on the specific utilization of features as you want

In [6]:
x_train = np.c_[X[:566,1],X[:566,3],X[:566,4],X[:566,5],X[:566,7]]
x_test = np.c_[X[566:,1],X[566:,3],X[566:,4],X[566:,5],X[566:,7]]
y_train = data_training[:, 8]
y_test = data_test[:, 8]

Reshaping dataset as for network flow

In [7]:
x_train = x_train.reshape(566, 5)
x_test = x_test.reshape(142, 5)
y_train = y_train.reshape(566,1)
y_test = y_test.reshape(142,1)

Early stopping the model training by validation loss

In [8]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

Set learning rate in Adam optimizer (this value has been already optimized)

In [9]:
opt = Adam(lr=1e-2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Artificial neural networks of 50 neurons on 1-hidden layer (this has been already optimized)

In [10]:
mdl = Sequential()
mdl.add(Dense(50, input_dim=5, activation='relu'))
mdl.add(Dense(1, activation='sigmoid'))
mdl.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])
hst = mdl.fit(x_train, y_train, batch_size=20, epochs=1500,
            validation_data=(x_test, y_test), callbacks=[es, mc])

Epoch 1/1500
29/29 [==============================] - 1s 9ms/step - loss: 0.4031 - accuracy: 0.8534 - val_loss: 0.3067 - val_accuracy: 0.8803
Epoch 2/1500
29/29 [==============================] - 0s 3ms/step - loss: 0.3058 - accuracy: 0.8728 - val_loss: 0.2805 - val_accuracy: 0.8944
Epoch 3/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.2803 - accuracy: 0.8816 - val_loss: 0.2447 - val_accuracy: 0.9014
Epoch 4/1500
29/29 [==============================] - 0s 3ms/step - loss: 0.2619 - accuracy: 0.8905 - val_loss: 0.2244 - val_accuracy: 0.9085
Epoch 5/1500
29/29 [==============================] - 0s 3ms/step - loss: 0.2514 - accuracy: 0.8975 - val_loss: 0.2343 - val_accuracy: 0.9155
Epoch 6/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.2338 - accuracy: 0.9011 - val_loss: 0.2000 - val_accuracy: 0.9225
Epoch 7/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.2230 - accuracy: 0.9028 - val_loss: 0.2046 - val_accuracy: 0.9225
Epoch 

Test accuracy at the end

In [11]:
print("\r Test accuracy : %.4f" % (mdl.evaluate(x_test, y_test)[1])*100)

5/5 [==============================] - 0s 3ms/step - loss: 0.1238 - accuracy: 0.9789
 Test accuracy : 0.9789
